In [2]:
import numpy as np 
import pandas as pd 
import fasttext
import bz2
import string
import csv
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import re
import random
from sklearn.ensemble import RandomForestClassifier
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
%matplotlib inline

In [3]:
import os
os.getcwd()

'/Users/dashadobrego/Downloads/Telegram Desktop'

## Fasttext approach

In [39]:
data = bz2.BZ2File("train.ft.txt.bz2")
data = data.readlines()
data = [x.decode('utf-8') for x in data]
len(data)

3600000

Initial train dataset consists of 3'600'000 Amazon reviews. When trying to proceed the whole sample, my kernel died. It took several iterations to reveal that my computer works best (=just works) at 370'000. Important credits for this hint: Zamira M.

In [40]:
sample_data = data[:370000]
len(sample_data)

370000

In [7]:
sample_data = pd.DataFrame(sample_data)
sample_data.to_csv("sample_train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [8]:
model = fasttext.train_supervised('sample_train.txt',label_prefix='__label__', thread=4, epoch = 10) 
#apply fasttext model to our data
print(model.labels, 'are the labels or targets the model is predicting')

['__label__2', '__label__1'] are the labels or targets the model is predicting


In [9]:
#test data 
test = bz2.BZ2File("test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]

In [10]:
sample_test = test[:40000] 
len(sample_test)

40000

In [11]:
#do the creaning
new = [w.replace('__label__2 ', '') for w in sample_test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

In [12]:
pred = model.predict(new)

In [13]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in sample_test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

In [14]:
print(roc_auc_score(labels, pred_labels))

0.9013146720906343


Result is pretty high - 90,1%.
Let's see how it goes on machine learning approach.

## Machine learning approach

In [15]:
data = pd.DataFrame(data)

In [ ]:
train['label'] = 0  #clear reviews from "label" strings and create a column to store labels
for i in range(0,len(train)):
    review_text = train.iloc[i,0]
    review_label = train.iloc[i,1]
    if review_text.startswith('__label__2 '):
        review_label = '__label__2'
    if review_text.startswith('__label__1 '):
        review_label = '__label__1' 
    train.iloc[i,1] = review_label

In [58]:
train.to_csv("train.csv", index=False, sep=' ', header=True) #store dataframe as csv

In [ ]:
test = pd.DataFrame(test) #same for test set
test['label'] = 0
s=0
for i in range(0,len(test)):
    review_text = test.iloc[i,0]
    review_label = test.iloc[i,1]
    if review_text.startswith('__label__2 '):
        review_label = '__label__2'
    if review_text.startswith('__label__1 '):
        review_label = '__label__1'
    test.iloc[i,1] = review_label

In [60]:
test.to_csv("test.csv", index=False, sep=' ', header=True)

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
test[:10]

,label,review
0,__label__2,Great CD: My lovely Pat has one of the GREAT v...
1,__label__2,One of the best game music soundtracks - for a...
2,__label__1,Batteries died within a year ...: I bought thi...
3,__label__2,"works fine, but Maha Energy is better: Check o..."
4,__label__2,Great for the non-audiophile: Reviewed quite a...
5,__label__1,DVD Player crapped out after one year: I also ...
6,__label__1,"Incorrect Disc: I love the style of this, but ..."
7,__label__1,DVD menu select problems: I cannot scroll thro...
8,__label__2,Unique Weird Orientalia from the 1930's: Exoti...
9,__label__1,"Not an ""ultimate guide"": Firstly,I enjoyed the..."


In [6]:
test_sample = test.sample(n = 40000, random_state=123).reset_index(drop=True) #take test sample of 40000
test_sample['label'].value_counts()

__label__1    20127
__label__2    19873
Name: label, dtype: int64

In [7]:
train_sample = train.sample(n = 370000, random_state=123).reset_index(drop=True) #take train sample of 370000
train_sample['label'].value_counts()

__label__1    185266
__label__2    184734
Name: label, dtype: int64

In [8]:
labels = {'__label__2': 1, '__label__1': 0}
train_sample['label'] = train_sample['label'].map(labels).astype(int) #turn label values to integers
test_sample['label'] = test_sample['label'].map(labels).astype(int)

In [9]:
def process_data(data):  #function that lowercases the data and removes punctuation
    data = data.str.lower() 
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

In [10]:
train_sample['clean'] = process_data(train_sample['review'])
test_sample['clean'] = process_data(test_sample['review'])

In [42]:
train_sample[:15]

,label,review,clean
0,0,this movie sucks: This movie supposedly about ...,this movie sucks this movie supposedly about m...
1,0,Good Entertainment: This program a well edited...,good entertainment this program a well edited ...
2,1,Does the job: This hamper does the job in my k...,does the job this hamper does the job in my ki...
3,0,"Buffett Mails it In: Being a huge Buffett fan,...",buffett mails it in being a huge buffett fan i...
4,1,Sharp as a razor... almost.: Wow! My replaceme...,sharp as a razor almost wow my replacement is ...
5,0,Great Show BUT Too Much $$$$$$: Although I've ...,great show but too much although ive been wai...
6,0,HORRIBLE DONT BUY: My child got this Frog land...,horrible dont buy my child got this frog land ...
7,1,"ok, purchased the wrong size: I bought this fo...",ok purchased the wrong size i bought this for ...
8,1,Guilt pleasure: I love Doris Day and really en...,guilt pleasure i love doris day and really enj...
9,0,Don't buy it-- READ THIS REVIEW: This movie is...,dont buy it read this review this movie is rea...


In [12]:
train_list = train_sample['clean'].values.tolist()
test_list = test_sample['clean'].values.tolist()

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dashadobrego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
vectorizer = TfidfVectorizer(max_features=2500, stop_words=stopwords.words('english')) #use TF-IDF
processed_train = vectorizer.fit_transform(train_list).toarray()
processed_test = vectorizer.fit_transform(test_list).toarray()

In [15]:
X_train = processed_train
y_train = train_sample['label']
X_test = processed_test
y_test = test_sample['label']

In [16]:
from xgboost import XGBClassifier
XGboost = XGBClassifier(n_jobs = 1, random_state=1234, learning_rate=0.9)
XGboost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1234,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [17]:
xgb = XGboost.predict_proba(X_test)

xgb

array([[0.78904146, 0.21095854],
       [0.54802406, 0.45197597],
       [0.7388567 , 0.2611433 ],
       ...,
       [0.8975423 , 0.10245771],
       [0.749894  , 0.250106  ],
       [0.4283334 , 0.5716666 ]], dtype=float32)

In [18]:
np.shape(xgb)

(40000, 2)

In [19]:
from sklearn.metrics import f1_score
xgb_whole = xgb[:,1]>=0.4 #define cutoff
xgb_int=xgb_whole.astype(np.int)

In [20]:
print(roc_auc_score(y_test, xgb_int))

0.5724893404514303


ROCAUC is very low compared to the fasttext results - only 58,22%. Could be possible to try different learning rates, however, I doubt that playing with LR can lead to 90%+ results.

## Neural network

Solution is based on these ideas: https://www.kaggle.com/muonneutrino/sentiment-analysis-with-amazon-reviews

In [29]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_sample['review'], train_sample['label'], random_state=432, test_size=0.2)

In [30]:
test_labels = test_sample['label']
test_texts = test_sample['review']

In [31]:
MAX_FEATURES = 11000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [32]:
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [33]:
vals = np.asarray(val_labels)
trains = np.asarray(train_labels)

In [34]:
def build_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
model = build_model()

In [35]:
model.fit(
    train_texts, 
    trains, 
    batch_size=128,
    epochs=2,
    validation_data=(val_texts, vals))

Train on 296000 samples, validate on 74000 samples
Epoch 1/2
296000/296000 [==============================] - 214s 724us/sample - loss: 0.2505 - binary_accuracy: 0.8947 - val_loss: 0.1994 - val_binary_accuracy: 0.9202
Epoch 2/2
296000/296000 [==============================] - 203s 686us/sample - loss: 0.1706 - binary_accuracy: 0.9343 - val_loss: 0.1912 - val_binary_accuracy: 0.9246


In [36]:
predictions = model.predict(test_texts)

In [37]:
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, predictions)))

ROC AUC score: 0.9769


###  In conclusion, NN approach provided the best result.